In [ ]:
!pip install --quiet --user "torch>=1.6, <1.9" "lightning-bolts" "pytorch-lightning>=1.3" "torchmetrics>=0.3" "torchvision" "torchtext" "torchaudio"
! pip install wandb

     |████████████████████████████████| 804.1 MB 2.2 kB/s 
     |████████████████████████████████| 316 kB 70.3 MB/s 
     |████████████████████████████████| 582 kB 64.3 MB/s 
     |████████████████████████████████| 408 kB 92.6 MB/s 
     |████████████████████████████████| 596 kB 74.6 MB/s 
     |████████████████████████████████| 136 kB 82.1 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 21.0 MB 215 kB/s 
     |████████████████████████████████| 23.2 MB 93.0 MB/s 
     |████████████████████████████████| 23.3 MB 1.1 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 272 kB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 17.4 MB 102 kB/s 
     |████████████████████████████████| 10.4 MB 70.3 MB/s 
     |████████████████████████████████| 8.0 MB 44.2 MB/s 
     |████████████████████████████████| 8.0 MB 36.6 MB/s 
     |█████

In [ ]:
!curl https://raw.githubusercontent.com/Ojasvi-97/DL_Final_Project_Spring_22/main/train_data.csv --output train.csv
!curl https://raw.githubusercontent.com/Ojasvi-97/DL_Final_Project_Spring_22/main/val_data.csv --output val.csv
!curl https://raw.githubusercontent.com/Ojasvi-97/DL_Final_Project_Spring_22/main/test_data.csv --output test.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.0M  100 11.0M    0     0  17.1M      0 --:--:-- --:--:-- --:--:-- 17.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3944k  100 3944k    0     0  3618k      0  0:00:01  0:00:01 --:--:-- 3615k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4199k  100 4199k    0     0  5041k      0 --:--:-- --:--:-- --:--:-- 5041k


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/Colab\ Notebooks/train_images.zip > /dev/null
!unzip gdrive/My\ Drive/Colab\ Notebooks/val_images.zip > /dev/null
!unzip gdrive/My\ Drive/Colab\ Notebooks/test_images.zip > /dev/null

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# !pip install --upgrade imgaug==0.2.6

In [ ]:
# !pip install --upgrade matplotlib==3.3.3

In [ ]:
import os
import pandas as pd
from PIL import Image
import random
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn.utils import shuffle
print(matplotlib.__version__)


from torchvision.io import read_image
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torch.nn as nn
from torch.nn.functional import cosine_similarity

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from torch.optim.lr_scheduler import OneCycleLR
import math

seed_everything(7)
NUM_WORKERS = int(os.cpu_count() - 1)
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
print(AVAIL_GPUS, BATCH_SIZE, NUM_WORKERS)

3.2.2


Global seed set to 7


1 256 3


In [ ]:
class HMDataset(Dataset):
    def __init__(self, data, image_dir):
        self.df = data
        self.image_dir = image_dir
        self.preprocess = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
        ])
        # preprocess function for text


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        ref_data = self.df.iloc[idx]
        ref_image = Image.open(os.path.join(self.image_dir, ref_data["location"].split("/")[1]))
        ref_image = self.preprocess(ref_image)
        ref_image /= 255.0
        ref_text = ref_data["detail_desc"]
        ref_label = ref_data["index_group_name"]
        art_id = ref_data["article_id"]
        
        pos_data = pd.DataFrame(self.df[self.df["index_group_name"] == ref_data["index_group_name"]].sample(n=1))
        pos_data = pos_data.iloc[0]
        pos_image = Image.open(os.path.join(self.image_dir, pos_data["location"].split("/")[1]))
        pos_image = self.preprocess(pos_image)
        pos_image /= 255.0
        
        pos_text = pos_data["detail_desc"]
        pos_label = pos_data["index_group_name"]
        
        neg_data = pd.DataFrame(self.df[self.df["index_group_name"] != ref_data["index_group_name"]].sample(n=1))
        neg_data = neg_data.iloc[0]
        neg_image = Image.open(os.path.join(self.image_dir, neg_data["location"].split("/")[1]))
        neg_image = self.preprocess(neg_image)
        neg_image /= 255.0
        
        neg_text = neg_data["detail_desc"]
        neg_label = neg_data["index_group_name"]
        
        attributes = {
            "ref_image": ref_image,
            "ref_text": ref_text,
            "ref_label": ref_label,
            "ref_article_id": art_id,
            "pos_image": pos_image,
            "pos_text": pos_text,
            "pos_label": pos_label,
            "neg_image": neg_image,
            "neg_text": neg_text,
            "neg_label": neg_label
        }
        
        return attributes

In [ ]:
class HMDataModule(pl.LightningDataModule):
    def __init__(self, train_path="", val_path="", test_path="", img_train="", img_val="", img_test="", batch_size= BATCH_SIZE):
        super().__init__()
        self.batch_size = batch_size

        #image dirs
        self.img_train = img_train
        self.img_val = img_val
        self.img_test = img_test

        #data 
        self.train_df, self.val_df, self.test_df = pd.read_csv(train_path), shuffle(pd.read_csv(val_path)), shuffle(pd.read_csv(test_path))
        

    def setup(self, stage=None):
        self.train_data = HMDataset(data=self.train_df, image_dir=self.img_train)
        self.val_data = HMDataset(data=self.val_df, image_dir=self.img_val)
        self.test_data = HMDataset(data=self.test_df, image_dir=self.img_test)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=NUM_WORKERS)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=NUM_WORKERS)

    

In [ ]:
class VisualEmbd(nn.Module):
    # resnet last 2 layers
    # dense layers
    # similarity
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.af = nn.ReLU()
        self.last_layer = nn.Linear(1000, 256)

        count=0
        for child in self.model.children():
            count+=1
            if count < 7:
                for param in child.parameters():
                    param.requires_grad = False
    
    

    def forward(self, x):
        x = self.model(x)
        x = self.last_layer(self.af(x))
        return x

In [ ]:
class LitVisualEmbd(LightningModule):
    def __init__(self, lr=0.01, margin=0.1):
        super().__init__()
        self.save_hyperparameters()
        self.model = VisualEmbd()
        self.margin = margin
        self.lr = lr
        self.pca = PCA(n_components=50)
        self.tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        return self.evaluate(batch, batch_idx, stage="train")

    def log_plot(self, embds, labels):

        df_subset = pd.DataFrame()

        pca_embd = self.pca.fit_transform(embds.cpu().detach().numpy())
        print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(self.pca.explained_variance_ratio_)))
        tsne_pca_results = self.tsne.fit_transform(pca_embd)

        
        df_subset['tsne-pca50-one'] = tsne_pca_results[:,0]
        df_subset['tsne-pca50-two'] = tsne_pca_results[:,1]

        df_subset["label"] = labels


        sns.scatterplot(
          x="tsne-pca50-one", y="tsne-pca50-two",
          hue="label",
          palette=sns.color_palette("hls", len(set(labels))),
          data=df_subset,
          legend="full",
          alpha=0.3)
        # self.log("t-sne", wandb.Image(fig))
        
        self.logger.experiment.log({"t-sne plot": plt})

        plt.close()
    
    def evaluate(self, batch, batch_idx, stage=None):
        ref_batch = batch["ref_image"]
        pos_batch = batch["pos_image"]
        neg_batch = batch["neg_image"]
        
        ref_embd = self(ref_batch)
        pos_embd = self(pos_batch)
        neg_embd = self(neg_batch)

        tml = torch.nn.TripletMarginLoss(margin=self.margin, p=2)
        loss = tml(ref_embd, pos_embd, neg_embd)
        
        if stage:
            self.log(f"{stage}_loss", loss.mean(), prog_bar=True)

            if batch_idx == 0:
              self.log_plot(ref_embd, batch["ref_label"])

        return loss.mean()

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, batch_idx, "val")

    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
#         steps_per_epoch = 40000 // BATCH_SIZE
        return {"optimizer": optimizer}

        


In [ ]:
hm_dm = HMDataModule(
    train_path="train.csv",
    val_path = "val.csv",
    test_path = "test.csv",
    img_train = "/content/train_images",
    img_val = "/content/val_images",
    img_test = "/content/test_images",
)

In [ ]:
model = LitVisualEmbd()
model.datamodule = hm_dm

# trainer = Trainer(
#           progress_bar_refresh_rate=10,
#           gpus=AVAIL_GPUS,
#           max_epochs=10,
#           logger=WandbLogger(project="final_project_visual"),
#           callbacks=[LearningRateMonitor(logging_interval="step")]
#       )
# trainer.fit(model, hm_dm)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# model_save_name = 'visual_triplet'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [ ]:

def test_plot(embds, labels):
      pca = PCA(n_components=50)
      tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300)

      df_subset = pd.DataFrame()

      pca_embd = pca.fit_transform(embds)
      print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca.explained_variance_ratio_)))
      tsne_pca_results = tsne.fit_transform(pca_embd)

      
      df_subset['tsne-pca50-one'] = tsne_pca_results[:,0]
      df_subset['tsne-pca50-two'] = tsne_pca_results[:,1]

      df_subset["label"] = labels



      sns.scatterplot(
        x="tsne-pca50-one", y="tsne-pca50-two",
        hue="label",
        palette=sns.color_palette("hls", len(set(labels))),
        data=df_subset,
        legend="full",
        alpha=0.3)
      
      plt.show()
      plt.close()
        

In [ ]:
from tqdm import tqdm

def test(test_loader):

  all_embds = []
  all_labels = []
  all_article_ids = []

  model2 = LitVisualEmbd()
  # model_save_name = 'visual_triplet'
  # path = F"/content/gdrive/My Drive/{model_save_name}"
  # model.load_state_dict(torch.load(path))
  model2.eval()

  for x in tqdm(test_loader):
    final_embds = model2(x["ref_image"])
    all_embds.append(final_embds.cpu().detach().numpy())

    all_labels.extend(x["ref_label"])
    all_article_ids.extend(x["ref_article_id"])


  print("!!!", len(all_embds))
  all_embs_con = np.concatenate(all_embds, axis=0)
  print("embds", all_embs_con.shape)
  # print("label", len(all_labels))

  # print("Plotting...")
  test_plot(all_embs_con, all_labels)

  # ret urn [all_embs_con, all_article_ids]



In [1]:


# test_path = "test.csv"
# test_images = "/content/test_images"
# test_df = pd.read_csv(test_path)
# print(len(test_df))
# test_data = HMDataset(data=test_df, image_dir=test_images)
# test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
# test_embds = test(test_loader=test_loader)
# print("finished test split")

# # ## train

# train_path = "train.csv"
# train_images = "/content/train_images"
# train_df = pd.read_csv(train_path)
# train_data = HMDataset(data=train_df, image_dir=train_images)
# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
# train_embds = test(test_loader=train_loader)
# print("finished train split")

# ## val

# val_path = "val.csv"
# val_images = "/content/val_images"
# val_df = pd.read_csv(val_path)
# val_data = HMDataset(data=val_df, image_dir=val_images)
# val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
# val_embds = test(test_loader=val_loader)
# print("finished val split")


df_test_embd = pd.DataFrame()

for i in range(test_embds[0].shape[1]):
  df_test_embd["feature_"+ str(i)] = test_embds[0][:, i]

df_test_embd["article_id"] = test_embds[1]


df_test_embd["article_id"] = df_test_embd["article_id"].apply(lambda x: x.item())

df_train_embd = pd.DataFrame()

for i in range(train_embds[0].shape[1]):
  df_train_embd["feature_"+ str(i)] = train_embds[0][:, i]

df_train_embd["article_id"] = train_embds[1]

df_train_embd["article_id"] = df_train_embd["article_id"].apply(lambda x: x.item())
df_val_embd = pd.DataFrame()

for i in range(val_embds[0].shape[1]):
  df_val_embd["feature_"+ str(i)] = val_embds[0][:, i]

df_val_embd["article_id"] = val_embds[1]
df_val_embd["article_id"] = df_val_embd["article_id"].apply(lambda x: x.item())
final_embds = pd.concat([df_test_embd, df_train_embd, df_val_embd], ignore_index=True)
final_embds.to_csv("visual_embeddings_final.csv")